In [1]:
import numpy as np
import pandas as pd
import pygame 
import math
import matplotlib.pyplot as plt
import json
import random

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Car:
    def __init__(self,initial_x,initial_y,time_elapsed):
        self.x = initial_x
        self.y = initial_y
        self.time_elapsed = time_elapsed
        
        self.speed = 0
        self.vector_angle = 0
        
        self.car_angle = 0
        
        self.steering_angle = 0
        self.accel = 0

        self.grip = 2
        self.steering_grip = 1.2 
        
        self.CAR_WIDTH = 20
        self.CAR_HEIGHT = 10
        self.CAR_COLOR = (0, 255, 0)
        
        self.max_angle = 80
        self.max_speed = 10
        
        self.friction_coef = 0.01
        
        self.accel_const = 0.1 
        self.brake_const = 0.05 
        self.steering_const = 10
        
    def calculate_all(self):
        self.car_angle = self.car_angle * (1-self.steering_grip*self.time_elapsed) + (self.steering_angle+self.car_angle) * self.steering_grip*self.time_elapsed
        self.vector_angle = self.vector_angle * (1-self.grip*self.time_elapsed) + self.car_angle * self.grip*self.time_elapsed
        
        potential_speed = self.speed+self.accel*self.time_elapsed
        
        if((potential_speed < self.max_speed)&(potential_speed > 0)):
            self.speed = self.speed+self.accel*self.time_elapsed
        
    def update_position(self):
        self.x = self.x + self.speed*np.cos(np.radians(self.vector_angle))
        self.y = self.y + self.speed*np.sin(np.radians(self.vector_angle))
        
    def speed_up(self):
        self.accel += self.accel_const/(self.speed+0.1)
        
    def brake(self):
        self.accel -= self.speed*self.brake_const
        
    def nothing(self):
        self.accel = 0

    def friction(self):
        self.speed -= self.speed*self.friction_coef
        
    def turn_right(self):
        if(self.steering_angle < self.max_angle):
            self.steering_angle += self.steering_const
        else:
            self.steering_angle = self.max_angle
            
    def reset_turn(self):
        if(np.abs(self.steering_angle)  < 1):
            self.steering_angle =0
        elif(self.steering_angle<0):
            self.steering_angle +=self.steering_const
        elif(self.steering_angle>0):
            self.steering_angle -=self.steering_const

    def turn_left(self):
        if(self.steering_angle > -self.max_angle):
            self.steering_angle -= self.steering_const
        else:
            self.steering_angle = -self.max_angle

    def draw(self, screen):
        car_surf = pygame.Surface((self.CAR_WIDTH, self.CAR_HEIGHT), pygame.SRCALPHA)
        car_surf.fill(self.CAR_COLOR)
        rotated_car = pygame.transform.rotate(car_surf, -self.car_angle)
        screen.blit(rotated_car, (self.x - rotated_car.get_width() / 2, self.y - rotated_car.get_height() / 2))
        
    def reset_all(self):
        self.x = 0
        self.y = 0
        
        self.speed = 0
        self.vector_angle = 0
        
        self.car_angle = 0
        
        self.steering_angle = 0
        self.accel = 0
        
    def get_vals(self):
        return self.x,self.y,self.car_angle
        

In [3]:
class extention_line:
    
    def __init__(self,angle,car:Car,outside_line,inside_line,screen):
      self.angle_from_car = angle
      self.car = car
      self.outside_line = outside_line
      self.inside_line = inside_line
      self.shortest_point = (0,0)
      self.screen = screen
      self.smallest = (0,(0,0))
    
    def update(self):
        self.x,self.y,self.angle = self.car.get_vals()
        self.point1,self.point2= self.generate_long_segment_from_point((self.x,self.y),self.angle+self.angle_from_car)
        self.shortest_point = self.shortest_intersection()
    
    def plot(self):
        pygame.draw.line(self.screen,(255,255,255),self.point1,self.point2)
        pygame.draw.circle(self.screen,(255,0,0),self.shortest_point,10)
        
    def find_dist(self,x,y):
        return np.sqrt((x-self.x)**2+(y-self.y)**2)
    
    def shortest_intersection(self):
        point_list = []
        for i in range(len(self.outside_line) - 1):
            p1 = self.outside_line[i]
            p2 = self.outside_line[i+1]
            
            intersection = self.line_segments_intersection(p1,p2,self.point1,self.point2)
            if (intersection is not None):
                point_list.append((self.find_dist(intersection[0],intersection[1]),intersection))
                
        for i in range(len(self.inside_line) - 1):
            p1 = self.inside_line[i]
            p2 = self.inside_line[i+1]
            
            intersection = self.line_segments_intersection(p1,p2,self.point1,self.point2)
            if (intersection is not None):
                point_list.append((self.find_dist(intersection[0],intersection[1]),intersection))
                
        if len(point_list) > 0:
            self.smallest = min(point_list, key=lambda i: i[0])

            return self.smallest[1]
        else:
            return self.smallest[1]           
    
    def generate_long_segment_from_point(self,point, angle_degrees, length=10000):
        x, y = point
        angle_radians = math.radians(angle_degrees)

        dx = math.cos(angle_radians) * length
        dy = math.sin(angle_radians) * length

        end_point = (x + dx, y + dy)
        return point, end_point

    def line_segments_intersection(self,p1, p2, p3, p4):
        x1, y1 = p1
        x2, y2 = p2
        x3, y3 = p3
        x4, y4 = p4

        denom = (y4 - y3)*(x2 - x1) - (x4 - x3)*(y2 - y1)

        if denom == 0:
            return None

        t = ((x4 - x3)*(y1 - y3) - (y4 - y3)*(x1 - x3)) / denom
        u = ((x2 - x1)*(y1 - y3) - (y2 - y1)*(x1 - x3)) / denom

        if 0 <= t <= 1 and 0 <= u <= 1:
            intersect_x = x1 + t * (x2 - x1)
            intersect_y = y1 + t * (y2 - y1)
            return (intersect_x, intersect_y)

        return None

In [4]:
class RaceTrack:
    def __init__(self,inner_points,outer_points):
        self.inner_points = inner_points
        self.outer_points = outer_points
        
        self.track_color = (50,50,50)
        self.border_color = (255,90,90)
        self.inner_color = (255,255,255)
    
    def plot_track(self,surface):
        pygame.draw.polygon(surface,self.track_color,self.outer_points,0)
        pygame.draw.polygon(surface,self.inner_color,self.inner_points,0)
        pygame.draw.lines(surface,self.border_color,True,self.inner_points,5)
        pygame.draw.lines(surface,self.border_color,True,self.outer_points,5)
        
    
        


In [5]:
def generate_track(centerline, width=100):
    inner = []
    outer = []
    n = len(centerline)
    for i in range(n):
        p1 = centerline[i - 1]
        p2 = centerline[i]
        p3 = centerline[(i + 1) % n]

        # Calculate the direction between p1 and p3 (smoothed)
        dx = p3[0] - p1[0]
        dy = p3[1] - p1[1]
        length = math.hypot(dx, dy)
        if length == 0:
            perp_dx, perp_dy = 0, 0
        else:
            perp_dx = -dy / length
            perp_dy = dx / length

        offset_x = perp_dx * width / 2
        offset_y = perp_dy * width / 2

        inner.append((p2[0] - offset_x, p2[1] - offset_y))
        outer.append((p2[0] + offset_x, p2[1] + offset_y))
    
    return inner, outer


In [6]:
centerline = [(792.9, 153.4),
(375.2, 153.4),
(259.5, 177.1),
(217.1, 235.8),
(258.1, 289.8),
(362.5, 288.8),
(711.1, 273.7),
(833.9, 278.4),
(935.5, 302.1),
(976.4, 356.1),
(973.6, 429.9),
(925.6, 475.4),
(790.1, 495.3),
(454.2, 444.1),
(342.7, 444.1),
(228.4, 477.3),
(191.7, 536.0),
(186.1, 608.0),
(196.0, 667.6),
(291.9, 725.4),
(444.4, 737.7),
(637.7, 738.6),
(797.2, 733.9),
(864.9, 685.6),
(912.9, 646.8),
(983.5, 631.6),
(1082.3, 652.5),
(1141.5, 685.6),
(1193.8, 729.2),
(1251.6, 756.6),
(1339.1, 734.8),
(1399.8, 706.4),
(1439.3, 668.6),
(1433.7, 609.8),
(1421.0, 568.2),
(1364.5, 535.0),
(1316.5, 534.1),
(1292.5, 531.3),
(1275.6, 481.1),
(1289.7, 388.3),
(1325.0, 315.3),
(1364.5, 223.5),
(1349.0, 186.6),
(1241.7, 175.2),
(1178.2, 178.0),
(1051.2, 168.6),
(887.5, 156.2),]

In [7]:
inner_points,outer_points = generate_track(centerline,100)

In [ ]:
pygame.init()

SCREEN_WIDTH = 1600
SCREEN_HEIGHT = 900

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
screen.fill((255,255,255))
pygame.display.set_caption('Racing Car Simulation')

car = Car(0,0,1/60)
track = RaceTrack(inner_points,outer_points)

line_list = []
for angle in range(0,360,20):
    print(angle)
    line_list.append(extention_line(angle,car,outer_points,inner_points,screen))
    
def update_and_plot():
    for line in line_list:
        line.update()
        line.plot()
        

# Main loop
running = True
while running:
    screen.fill((0, 0, 0))  # Clear the screen

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    
    track.plot_track(screen)
    
    update_and_plot()
    

    keys = pygame.key.get_pressed()
    if keys[pygame.K_UP]:
        car.speed_up()
    elif keys[pygame.K_DOWN]:
        car.brake()
    else:
        car.nothing()
        
    if keys[pygame.K_LEFT]:
        car.turn_left()
    elif keys[pygame.K_RIGHT]:
        car.turn_right()
    else:
        car.reset_turn()
        
    if keys[pygame.K_r]:
        car.reset_all()

    car.friction()
    car.calculate_all()
    car.update_position()

    car.draw(screen)

    pygame.display.update()

    pygame.time.Clock().tick(60) 

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340


2025-05-18 17:31:13.839 python[89911:23982074] +[IMKClient subclass]: chose IMKClient_Modern
2025-05-18 17:31:13.839 python[89911:23982074] +[IMKInputSession subclass]: chose IMKInputSession_Modern


KeyboardInterrupt: 